# PMML - Predictive Model Markup Language

## Motivación

Como se vió en la clase anterior *Machine Learning Distribuido con PySpark*, hoy en día no hay una sola framework *probada en batalla* para machine learning en tiempo real, es decir baja latencia y alta cantidad de mensajes por unidad de tiempo.

Dicho esto, lo mejor que podemos hacer es construir la solución con los componentes que tenemos disponibles.

En esta clase veremos algunas maneras diferentes de mitigar este problema y poder **servir modelos en tiempo real y batch** con Spark con baja latencia y alta cantidad de mensajes por unidad de tiempo.

## PMML

<center>
  <img src="https://upload.wikimedia.org/wikipedia/en/8/80/PMML_Logo.png" alt="PMML Logo" width="800"/>
</center>

**PMML** o *Predictive Model Markup Language* por sus siglas en inglés es un archivo de definición para modelos predictivos en formato XML. El objetivo es la estandarización en formato e intercambio de modelos predictivos. Este estandar define una manera para que las aplicaciones analíticas puedan describir los modelos producidos por machine learning.

### Archivo PMML

Más alla de que el archivo no esta pensado para que sea leido por humanos, sino para el intercambio entre frameworks y máquinas, entender las diferentes partes del archivo nos puede ayudar a comprender como funciona su intercambio.

*NOTA: Se escriben los nombres en inglés ya que así se encontrarán en el archivo*

- **Header o Cabecera**: Contiene información general del documento PMML como el copyright, descripción, tiempo de generación, etc.
- **Data Dictionary o Diccionario de Datos**: Contiene definiciones para todas las variables utilizadas por el modelo
- **Data Transformations o Transformaciones de Datos**: Permite la transformación de data original al input deseable para el modelo
  - Normalización: mapear valores a numeros (continuos o discretos)
  - Discretización: mapear valores continuos a discretos
  - Mapeo de Valores: mapear valores discretos a discretos
  - Funciones: (custom o integradas) derivar un valor como resultado de una función.
  - Agregaciones: resumir o recolectar grupos de valores.
- **Model o Modelo**: Contiene información sobre el modelo como nombre, funcion, algoritmo, funcion de activación y numero de capas.
- **Mining Schema**: Una lista de todas las variables utilizadas en el modelo. Puede ser un subset del **Diccionario de Datos** pero este contiene información más detallada.
- **Targets**: Procesado posterior al resultado del modelo, como pasar de un resultado numérico a una clasificación.
- **Output**: Este elemento puede ser utilizado para nombrar todos los outputs esperados del modelo.

## Spark

### Dependencias

Aquí se instalan las dependencias y descargan los archivos necesarios para correr este colab

In [1]:
!pip install pyspark==3.2.0 pyspark2pmml openscoring==0.5.0
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
!wget https://repo1.maven.org/maven2/org/jpmml/pmml-sparkml/2.2.0/pmml-sparkml-2.2.0.jar
!wget https://github.com/jpmml/jpmml-sparkml/releases/download/2.2.0/pmml-sparkml-example-executable-2.2.0.jar
!cp *.jar /usr/local/lib/python3.7/dist-packages/pyspark/jars
!wget https://github.com/openscoring/openscoring/releases/download/2.1.0/openscoring-server-executable-2.1.0.jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 19.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805897 sha256=3bf1228ef0b82e2afe42d2418d9176a2d2caa86fc1bb206d873dccf86673b94e
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
  Created wheel for openscoring: filename=openscoring-0.5.0-py3-none-any.whl size=15447 sha256=828074967167a843980799e0c3c1d0d26e4ae32459600c4aa24e540313985fca
  Stored in directory: /root/.cache/pip/wheels/9e/dc/6d/f693595af7fc650ef112694173f372d1e9dcc692311cd9d6c1
  Created wheel for pyspark2pmml: filename=pyspark2pmml-0.5.1-py3-none-any.whl size=2397 sha256=fb353c8c4f7b6cb1688b96ea01ed402914e44ebfe87b7696aa172ed788be8772
  Stored in director

### Imports

In [2]:
import os
from openscoring import Openscoring

from pyspark2pmml import PMMLBuilder
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.types import StructType, DoubleType, StringType
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import (
    MinMaxScaler,
    VectorAssembler,
    OneHotEncoder,
    StringIndexer,
    IndexToString
)

### Creando el cluster de Spark con las dependencias instaladas

Como se ve aquí, se estan agregando archivos JAR. Los JAR son paquetes de codigo compilado de Java. Para poder agregar dependencias de Java a Spark, es necesario descargar este tipo de archivos y agregarlos al cluster. Hay muchas maneras de hacer esto. En este caso lo que se esta haciendo es agregarlos como configuración.

Adicionalmente, se crea el cluster de Spark con `local[*]` para que el cluster decida la cantidad de threads que necesita para correr el notebook.

In [3]:
jars = [
  'pmml-sparkml-example-executable-2.2.0.jar',
  'pmml-sparkml-2.2.0.jar',
  '/content/pmml-sparkml-example-executable-2.2.0.jar',
  '/content/pmml-sparkml-2.2.0.jar'
]

joined_jars = ",".join(jars)
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--packages org.jpmml:pmml-sparkml:2.0.0 ' + \
  f'--master local[*] --jars {joined_jars} ' + \
  'pyspark-shell'

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .config('spark.jars', joined_jars) \
    .appName("PMML") \
    .getOrCreate()
sc = spark.sparkContext

### Importando el dataset

En el siguiente bloque se define el schema. En la mayoría de los casos esto no es necesario, pero como las columnas del dataset `iris.csv` tienen puntos en los nombres: `sepal.width` Spark entiende que es un `Struct` o un objeto y trata de descomponerlo. Como no puede, este falla. Lo que hacemos para solucionar esto es cambiarle el nombre agregando *backticks* (el siguiente caracter: `)

In [4]:
iris_schema = StructType().add('sepal.length', DoubleType()) \
  .add('sepal.width', DoubleType()) \
  .add('petal.length', DoubleType()) \
  .add('petal.width', DoubleType()) \
  .add('variety', StringType())

# renaming columns to remove dot for better compatibility
iris_df = spark.read.format('csv') \
  .schema(iris_schema) \
  .option('header', 'true') \
  .load('iris.csv') \
  .select(
      col('`sepal.width`').alias('sepal_width'),
      col('`sepal.length`').alias('sepal_length'),
      col('`petal.width`').alias('petal_width'),
      col('`petal.length`').alias('petal_length'),
      col('variety')
    )
iris_df.show()
iris_df.printSchema()

+-----------+------------+-----------+------------+-------+
|sepal_width|sepal_length|petal_width|petal_length|variety|
+-----------+------------+-----------+------------+-------+
|        3.5|         5.1|        0.2|         1.4| Setosa|
|        3.0|         4.9|        0.2|         1.4| Setosa|
|        3.2|         4.7|        0.2|         1.3| Setosa|
|        3.1|         4.6|        0.2|         1.5| Setosa|
|        3.6|         5.0|        0.2|         1.4| Setosa|
|        3.9|         5.4|        0.4|         1.7| Setosa|
|        3.4|         4.6|        0.3|         1.4| Setosa|
|        3.4|         5.0|        0.2|         1.5| Setosa|
|        2.9|         4.4|        0.2|         1.4| Setosa|
|        3.1|         4.9|        0.1|         1.5| Setosa|
|        3.7|         5.4|        0.2|         1.5| Setosa|
|        3.4|         4.8|        0.2|         1.6| Setosa|
|        3.0|         4.8|        0.1|         1.4| Setosa|
|        3.0|         4.3|        0.1|  

### Preprocesamiento de los datos para el modelo

El siguiente bloque puede considerarse que ejecuta acciones redundantes, pero fue escrito asi intencionalmente para mostrar que puede seguir modificandose el `DataFrame` original y no necesariamente tiene que ser todo trabajado dentro de un vector proveniente del `VectorAssembler`

In [5]:
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
labels = ['variety']

iris_df.show()

df = iris_df

# UDF for converting column type from vector to double type
unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())

for feature in features:
  # VectorAssembler Transformation - Converting column to vector type
  vector_feature = f'{feature}_vect'
  assembler = VectorAssembler(inputCols=[feature], outputCol=vector_feature)

  # MinMaxScaler Transformation
  scaled_feature = f'{feature}_scaled'
  scaler = MinMaxScaler(inputCol=vector_feature, outputCol=scaled_feature)

  # Pipeline of VectorAssembler and MinMaxScaler
  pipeline = Pipeline(stages=[assembler, scaler])

  # Fitting pipeline on dataframe
  df = pipeline.fit(df) \
    .transform(df) \
    .withColumn(scaled_feature, unlist(scaled_feature)) \
    .drop(vector_feature)

df.show()
iris_df_scaled = df

+-----------+------------+-----------+------------+-------+
|sepal_width|sepal_length|petal_width|petal_length|variety|
+-----------+------------+-----------+------------+-------+
|        3.5|         5.1|        0.2|         1.4| Setosa|
|        3.0|         4.9|        0.2|         1.4| Setosa|
|        3.2|         4.7|        0.2|         1.3| Setosa|
|        3.1|         4.6|        0.2|         1.5| Setosa|
|        3.6|         5.0|        0.2|         1.4| Setosa|
|        3.9|         5.4|        0.4|         1.7| Setosa|
|        3.4|         4.6|        0.3|         1.4| Setosa|
|        3.4|         5.0|        0.2|         1.5| Setosa|
|        2.9|         4.4|        0.2|         1.4| Setosa|
|        3.1|         4.9|        0.1|         1.5| Setosa|
|        3.7|         5.4|        0.2|         1.5| Setosa|
|        3.4|         4.8|        0.2|         1.6| Setosa|
|        3.0|         4.8|        0.1|         1.4| Setosa|
|        3.0|         4.3|        0.1|  

En el próximo paso, se define el `StringIndexer` para mapear valores categoricos a numéricos

In [6]:
indexers = [StringIndexer(inputCol=label, outputCol=f'{label}_numeric') \
            .fit(iris_df_scaled) for label in labels]

pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(iris_df_scaled).transform(iris_df_scaled)
df_indexed.show()

+-----------+------------+-----------+------------+-------+-------------------+------------------+-------------------+------------------+---------------+
|sepal_width|sepal_length|petal_width|petal_length|variety|sepal_length_scaled|sepal_width_scaled|petal_length_scaled|petal_width_scaled|variety_numeric|
+-----------+------------+-----------+------------+-------+-------------------+------------------+-------------------+------------------+---------------+
|        3.5|         5.1|        0.2|         1.4| Setosa|              0.222|             0.625|              0.068|             0.042|            0.0|
|        3.0|         4.9|        0.2|         1.4| Setosa|              0.167|             0.417|              0.068|             0.042|            0.0|
|        3.2|         4.7|        0.2|         1.3| Setosa|              0.111|               0.5|              0.051|             0.042|            0.0|
|        3.1|         4.6|        0.2|         1.5| Setosa|              0.0

### Training y testing set (y validation cuando es posible)

Finalmente, se divide en el test de training y testing.

Recuerden que cuando peudan siempre deberan agregar el test de validación para evitar *overfittear* al set de testing.

In [7]:
(training, testing) = df_indexed.select('sepal_length_scaled',
                                        'sepal_width_scaled',
                                        'petal_length_scaled',
                                        'petal_width_scaled',
                                        'variety_numeric') \
                                        .withColumn('label',
                                                    col('variety_numeric')) \
                                        .drop('variety_numeric') \
                                        .randomSplit([0.7, 0.3])

df_indexed.show(5)
training.show(5)
testing.show(5)

+-----------+------------+-----------+------------+-------+-------------------+------------------+-------------------+------------------+---------------+
|sepal_width|sepal_length|petal_width|petal_length|variety|sepal_length_scaled|sepal_width_scaled|petal_length_scaled|petal_width_scaled|variety_numeric|
+-----------+------------+-----------+------------+-------+-------------------+------------------+-------------------+------------------+---------------+
|        3.5|         5.1|        0.2|         1.4| Setosa|              0.222|             0.625|              0.068|             0.042|            0.0|
|        3.0|         4.9|        0.2|         1.4| Setosa|              0.167|             0.417|              0.068|             0.042|            0.0|
|        3.2|         4.7|        0.2|         1.3| Setosa|              0.111|               0.5|              0.051|             0.042|            0.0|
|        3.1|         4.6|        0.2|         1.5| Setosa|              0.0

### Entrenamiento del modelo

En el siguiente paso se crea el modelo. Si se fijan, también estamos creando un `IndexToString` que ejecuta el proceso inverso del `StringIndexer`. Cuando tengo el resultado del modelo nunca voy a saber si `0` corresponde a `Setosa` o `Virginica`. Para esto sirve el `IndexToString`.

Estamos tomando un `RandomForestClassifier` que es uno de los algoritmos más robustos dentro del machine learning tabular.

In [8]:
# We can also use the multinomial family for binary classification
index_to_class = [IndexToString(inputCol='prediction', outputCol='prediction_class', labels=i.labels) for i in indexers]

model = Pipeline(stages=[
  VectorAssembler(
    inputCols=[f'{feature}_scaled' for feature in features],
    outputCol='features'),
  RandomForestClassifier(numTrees=125, maxDepth=5)
] + index_to_class)

# Fit the model
model = model.fit(training)

training_predictions = model.transform(training)
testing_predictions = model.transform(testing)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

training_accuracy = evaluator.evaluate(training_predictions)
testing_accuracy = evaluator.evaluate(testing_predictions)

print(f'Train Accuracy: {str(training_accuracy)}')
print(f'Train Error: {1.0 - training_accuracy}')

print(f'Test Accuracy: {str(testing_accuracy)}')
print(f'Test Error: {1.0 - testing_accuracy}')

training_predictions.show(5)
testing_predictions.show(5)

Train Accuracy: 1.0
Train Error: 0.0
Test Accuracy: 0.9615384615384616
Test Error: 0.038461538461538436
+-------------------+------------------+-------------------+------------------+-----+--------------------+---------------+-----------------+----------+----------------+
|sepal_length_scaled|sepal_width_scaled|petal_length_scaled|petal_width_scaled|label|            features|  rawPrediction|      probability|prediction|prediction_class|
+-------------------+------------------+-------------------+------------------+-----+--------------------+---------------+-----------------+----------+----------------+
|                0.0|             0.417|              0.017|               0.0|  0.0|[0.0,0.417,0.017,...|[125.0,0.0,0.0]|    [1.0,0.0,0.0]|       0.0|          Setosa|
|              0.028|             0.417|              0.051|             0.042|  0.0|[0.028,0.417,0.05...|[125.0,0.0,0.0]|    [1.0,0.0,0.0]|       0.0|          Setosa|
|              0.028|               0.5|           

## **ATENCIÓN: PMML**

Persistir el modelo es tan facil como hacer lo siguiente:

In [9]:
pmmlBuilder = PMMLBuilder(sc, training, model)
pmmlBuilder.buildFile("RandomForestIris.pmml")

'/content/RandomForestIris.pmml'

### Servir los modelos

Con el próximo comando iniciamos un server llamado **OpenScoring**. Este es un servidor en Java que permite recibir el modelo como un request, y luego servirlo para poder hacer predicciones en tiempo real.

OpenScoring no es el único servidor que permite servir modelos en tiempo real. También existe **MLeap**. Esta, aún más popular, es una librería open source para desplegar pipelines de datos y algoritmos sin necesidad de escribir más de 10 líneas de código. MLeap permite desplegar pipelines de Spark y Scikit-learn. Para este caso, MLeap utiliza un servidor Spring, también en Java, con la ejecución core en Scala.

In [10]:
!nohup java -jar /content/openscoring-server-executable-2.1.0.jar --port 8081 &
!sleep 10

nohup: appending output to 'nohup.out'


### Data Locallity o Localidad de la Data

Las personas que diseñaron Spark notaron que es más costoeficiente "mover los cómputos" que "mover la data". Es decir, es más barato ejecutar los computos donde esta la data que mover la data a donde esta el computo. Por eso, no solamente Spark es procesamiento distribuido, sino que usa un patrón crucial para su funcionamiento óptimo. Este es, tener en cuenta la **Localidad de los datos**. Esto significa que los procesamientos que se envían al cluster de Spark, deben intentar poder ser performados por las máquinas en donde la data esta y evitar el *shuffling* (que los datos de una maquina termine en otra, que vimos que es costoso).

Es importante tener esto en cuenta al momento de diseñar un sistema utilizando las tecnologías vistas en este colab. Se podría pensar en una arquitectura con los modelos desplegados en la misma máquina donde esta la data, de esta manera las consultas no saldrían de esta y sería extremadamente rápido, a pesar de que fuera HTTP.

Los invito a considerar diferentes opciones y conversarlas en el discord.

In [11]:
!curl -X PUT --data-binary @RandomForestIris.pmml -H "Content-type: text/xml" http://localhost:8081/openscoring/model/RandomForestIris

{
  "id" : "RandomForestIris",
  "miningFunction" : "classification",
  "summary" : "Ensemble model",
  "properties" : {
    "created.timestamp" : "2023-08-07T22:54:00.861+00:00",
    "accessed.timestamp" : null,
    "file.size" : 289456,
    "file.checksum" : "3e832ab7062f2a490d8ccf6064c700e4d90e39cfd1bba6b1bd00b95f81cd6053",
    "model.version" : null
  },
  "schema" : {
    "inputFields" : [ {
      "id" : "sepal_length_scaled",
      "dataType" : "double",
      "opType" : "continuous"
    }, {
      "id" : "sepal_width_scaled",
      "dataType" : "double",
      "opType" : "continuous"
    }, {
      "id" : "petal_length_scaled",
      "dataType" : "double",
      "opType" : "continuous"
    }, {
      "id" : "petal_width_scaled",
      "dataType" : "double",
      "opType" : "continuous"
    } ],
    "targetFields" : [ {
      "id" : "label",
      "dataType" : "double",
      "opType" : "categorical",
      "values" : [ "0.0", "1.0", "2.0" ]
    } ],
    "outputFields" : [ {
   

In [12]:
from json import dump

evaluation = {
    'id': 'record-001',
    'arguments': {'sepal_length_scaled': 0.0, 'sepal_width_scaled': 0.417,
             'petal_length_scaled': 0.017, 'petal_width_scaled': 0.0}
}

with open('test-data.json', 'w') as f:
  dump(evaluation, f)

In [13]:
!curl -X POST --data-binary @test-data.json -H "Content-type: application/json" http://localhost:8081/openscoring/model/RandomForestIris

{
  "id" : "record-001",
  "results" : {
    "label" : 0.0,
    "prediction" : 0.0,
    "probability(0)" : 1.0,
    "probability(1)" : 0.0,
    "probability(2)" : 0.0
  }
}

## Menciones honorables

- Tensorflow Extended
- Tensorflow Serving
- KubeFlow
- ONNX
- TensorRT